<br>
@Author: Ayush Prajapati<br>
@Date: 02-09-2024 <br>
@Last Modified by: Ayush Prajapati<br>
@Last Modified time: 03-09-2024 <br>
@Title : Spark Commands to Perform Operations on Covid Dataset<br><br>

In [31]:
 from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("COVID-19 Analysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-09-03 10:49:18,308 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Q1: To find out the death percentage locally and globally<br>

In [3]:
## Importing data
country_wise_latest_df = spark.read.csv("file:/home/hdoop/datasets/country_wise_latest.csv", header=True, inferSchema=True)


In [5]:
country_wise_latest_df.show()

+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|     Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|        Afghanistan|    36263|  1269|    25198|  9796|      106|        10|           18|               3.5|                69.49|                  5.04|              35526|          737|             2.07|Eastern Mediterra...|
|            Albania|     4880|   144|     2745|  1991|      117|         6|           6

In [6]:
country_wise_latest_df.printSchema()

root
 |-- Country/Region: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- Deaths / 100 Cases: double (nullable = true)
 |-- Recovered / 100 Cases: double (nullable = true)
 |-- Deaths / 100 Recovered: double (nullable = true)
 |-- Confirmed last week: integer (nullable = true)
 |-- 1 week change: integer (nullable = true)
 |-- 1 week % increase: double (nullable = true)
 |-- WHO Region: string (nullable = true)



In [8]:
country_wise_latest_df.createOrReplaceTempView("country_wise_latest_df")

In [12]:
# Globally

spark.sql('''SELECT 
(ROUND((SUM(Deaths) * 100.0)/ SUM(Confirmed),3)) AS Global_Death_Percentage
FROM country_wise_latest_df;''').show()

+-----------------------+
|Global_Death_Percentage|
+-----------------------+
|                  3.969|
+-----------------------+



In [18]:
# Locally

spark.sql('''SELECT `Country/Region`,
(ROUND(((SUM(Deaths) * 100.0)/ SUM(Confirmed)),3)) AS Death_Percentage
FROM country_wise_latest_df
GROUP BY `Country/Region`
HAVING `Country/Region` = 'India';''').show()


+--------------+----------------+
|Country/Region|Death_Percentage|
+--------------+----------------+
|         India|           2.257|
+--------------+----------------+



## Q2: To find out the infected population percentage locally and globally<br>

In [32]:
## Importing data 
worldometer_data_df = spark.read.csv("file:/home/hdoop/datasets/worldometer_data.csv", header=True, inferSchema=True)
worldometer_data_df.printSchema()

root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)



In [22]:
worldometer_data_df.createOrReplaceTempView("worldometer_data_df")

In [23]:
# Globally

spark.sql('''SELECT 
    SUM(CAST(Population AS BIGINT)) AS Global_Population,
    SUM(TotalCases) AS TotalCases,
    (((SUM(TotalCases) * 100.0)/ SUM(CAST(Population AS BIGINT)))) AS Global_Infection_Rate
FROM 
    worldometer_data_df;''').show()

+-----------------+----------+---------------------+
|Global_Population|TotalCases|Global_Infection_Rate|
+-----------------+----------+---------------------+
|       6326421290|  19169166|     0.30300173069884|
+-----------------+----------+---------------------+



In [30]:
# Locally

spark.sql('''SELECT 
    Continent, `Country/Region`,
    (ROUND(((TotalCases * 100.0) / Population),2)) AS Country_Infection_Rate
    FROM worldometer_data_df
    WHERE `Country/Region` = 'India';''').show()

+---------+--------------+----------------------+
|Continent|Country/Region|Country_Infection_Rate|
+---------+--------------+----------------------+
|     Asia|         India|                  0.15|
+---------+--------------+----------------------+



## Q3: To find out the countries with the highest infection rates<br>

In [37]:
spark.sql('''SELECT
             Continent, `Country/Region`,
             (((TotalCases * 100.0) / Population)) AS Country_Infection_Rate
             FROM worldometer_data_df
             ORDER BY Country_Infection_Rate DESC
             LIMIT 10;''').show()

+-------------+--------------+----------------------+
|    Continent|Country/Region|Country_Infection_Rate|
+-------------+--------------+----------------------+
|         Asia|         Qatar|        3.992157575045|
|South America| French Guiana|        2.714564857959|
|         Asia|       Bahrain|        2.513023907975|
|       Europe|    San Marino|        2.059638163710|
|South America|         Chile|        1.916481022828|
|North America|        Panama|        1.652703989233|
|         Asia|        Kuwait|        1.637844316754|
|         Asia|          Oman|        1.576904396373|
|North America|           USA|        1.519386296052|
|       Europe|  Vatican City|        1.498127340824|
+-------------+--------------+----------------------+



## Q4: To find out the countries and continents with the highest death counts<br>

In [39]:
# By Countries

spark.sql('''SELECT
             `Country/Region`, TotalDeaths
             FROM worldometer_data_df
             ORDER BY TotalDeaths DESC
             LIMIT 3;''').show()

+--------------+-----------+
|Country/Region|TotalDeaths|
+--------------+-----------+
|           USA|     162804|
|        Brazil|      98644|
|        Mexico|      50517|
+--------------+-----------+



In [41]:
# By Continents

spark.sql('''SELECT
             Continent, 
             SUM(TotalDeaths) AS Total_Continent_Deaths
             FROM worldometer_data_df
             GROUP BY Continent
             ORDER BY Total_Continent_Deaths DESC
             LIMIT 3;''').show()

+-------------+----------------------+
|    Continent|Total_Continent_Deaths|
+-------------+----------------------+
|North America|                229855|
|       Europe|                205232|
|South America|                154885|
+-------------+----------------------+



## Q5: Average number of deaths by day (Continents and Countries)<br>

In [44]:
## Importing data
covid_19_clean_df = spark.read.csv("file:/home/hdoop/datasets/covid_19_clean_complete.csv", header=True, inferSchema=True)
covid_19_clean_df.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)



In [45]:
covid_19_clean_df.createOrReplaceTempView("covid_19_clean_df")

In [79]:
# By Countries

spark.sql('''SELECT 
             `Date`, 
             `Country/Region`,
             (AVG(Deaths)) AS Average_Death 
             FROM covid_19_clean_df
             GROUP BY `Date`, `Country/Region`
             ORDER BY RAND();''').show()


+----------+-------------------+-------------+
|      Date|     Country/Region|Average_Death|
+----------+-------------------+-------------+
|2020-02-22|            Estonia|          0.0|
|2020-06-20|        Switzerland|       1956.0|
|2020-04-04|      Cote d'Ivoire|          1.0|
|2020-04-24|            Ireland|       1014.0|
|2020-07-13| Dominican Republic|        903.0|
|2020-07-17|Trinidad and Tobago|          8.0|
|2020-02-18|            Somalia|          0.0|
|2020-06-12|            Finland|        325.0|
|2020-05-19|           Zimbabwe|          4.0|
|2020-01-24|             Brunei|          0.0|
|2020-03-04|             Canada|          0.0|
|2020-03-28|             Russia|          4.0|
|2020-07-23|     United Kingdom|       4149.0|
|2020-06-26|               Mali|        113.0|
|2020-03-31|        Switzerland|        433.0|
|2020-05-05|          Nicaragua|          5.0|
|2020-04-27|            Belgium|       7207.0|
|2020-07-26|             Panama|       1294.0|
|2020-06-19| 

In [80]:
## By Continents

spark.sql('''SELECT 
            `Date`, 
            `WHO Region`,
            ROUND(AVG(Deaths),2) AS Average_Death
            FROM covid_19_clean_df
            GROUP BY `Date`, `WHO Region`;''').show()


+----------+--------------------+-------------+
|      Date|          WHO Region|Average_Death|
+----------+--------------------+-------------+
|2020-03-06|     South-East Asia|          0.1|
|2020-06-12|              Europe|      2347.46|
|2020-06-22|     Western Pacific|       133.33|
|2020-06-27|              Europe|      2460.16|
|2020-01-25|              Africa|          0.0|
|2020-02-10|     South-East Asia|          0.0|
|2020-04-20|              Africa|        14.69|
|2020-05-28|Eastern Mediterra...|       537.09|
|2020-06-16|            Americas|      4553.02|
|2020-06-29|     South-East Asia|       2159.3|
|2020-07-02|     South-East Asia|       2324.2|
|2020-02-09|            Americas|          0.0|
|2020-02-14|              Europe|          0.0|
|2020-02-17|            Americas|          0.0|
|2020-02-28|     South-East Asia|          0.0|
|2020-03-01|              Africa|          0.0|
|2020-06-17|Eastern Mediterra...|       842.27|
|2020-06-18|            Americas|       

## Q6: Average of cases divided by the number of population of each country (TOP 10)<br>

In [93]:
spark.sql('''SELECT
             `Country/Region`,
             ROUND(AVG((TotalCases * 1.0) / Population), 3) AS Average_Infection_Rate
             FROM worldometer_data_df
             GROUP BY `Country/Region`
             LIMIT 10;''').show()


+----------------+----------------------+
|  Country/Region|Average_Infection_Rate|
+----------------+----------------------+
|            Chad|                 0.000|
|          Russia|                 0.006|
|        Paraguay|                 0.001|
|           Macao|                 0.000|
|             CAR|                 0.001|
|           Yemen|                 0.000|
|Turks and Caicos|                 0.003|
|         Senegal|                 0.001|
|          Sweden|                 0.008|
|      Cabo Verde|                 0.005|
+----------------+----------------------+

